<a href="https://colab.research.google.com/github/DDKK1234/DDKK1234.github.io/blob/main/A1BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas bokeh geopandas

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving reviews_202106.csv to reviews_202106.csv
Saving reviews_202107.csv to reviews_202107.csv
Saving reviews_202108.csv to reviews_202108.csv
Saving reviews_202109.csv to reviews_202109.csv
Saving reviews_202110.csv to reviews_202110.csv
Saving reviews_202111.csv to reviews_202111.csv
Saving reviews_202112.csv to reviews_202112.csv
Saving sales_202106.csv.zip to sales_202106.csv.zip
Saving sales_202107.csv.zip to sales_202107.csv.zip
Saving sales_202108.csv.zip to sales_202108.csv.zip
Saving sales_202109.csv.zip to sales_202109.csv.zip
Saving sales_202110.csv.zip to sales_202110.csv.zip
Saving sales_202111.csv.zip to sales_202111.csv.zip
Saving sales_202112.csv.zip to sales_202112.csv.zip
Saving stats_crashes_202106_overview.csv to stats_crashes_202106_overview.csv
Saving stats_crashes_202107_overview.csv to stats_crashes_202107_overview.csv
Saving stats_crashes_202108_overview.csv to stats_crashes_202108_overview.csv
Saving stats_crashes_202109_overview.csv to stats_crashes_202109_o

In [ ]:
import zipfile
import os

for file_name in uploaded.keys():
    if file_name.endswith('.zip'):
        try:
            with zipfile.ZipFile(file_name, 'r') as zip_ref:
                zip_ref.extractall()
                print(f"{file_name}")
        except zipfile.BadZipFile:
            print(f"F {file_name}")

sales_202106.csv.zip
sales_202107.csv.zip
sales_202108.csv.zip
sales_202109.csv.zip
sales_202110.csv.zip
sales_202111.csv.zip
sales_202112.csv.zip
stats_ratings_202107_country.csv.zip
stats_ratings_202106_country.csv.zip
stats_ratings_202108_country.csv.zip
stats_ratings_202109_country.csv.zip
stats_ratings_202110_country.csv.zip
stats_ratings_202112_country.csv.zip
stats_ratings_202111_country.csv.zip


In [ ]:
import pandas as pd

#sales1.1
sales_files = [
    "sales_202106.csv", "sales_202107.csv", "sales_202108.csv",
    "sales_202109.csv", "sales_202110.csv", "sales_202111.csv", "sales_202112.csv"
]

sales_data = {}
for file in sales_files:
    try:
        print(f"Loading {file}...")
        df = pd.read_csv(file, encoding='utf-8')
        sales_data[file] = df
    except UnicodeDecodeError:
        print(f"Failed to load {file} with UTF-8. Trying ISO-8859-1...")
        df = pd.read_csv(file, encoding='ISO-8859-1')
        sales_data[file] = df

for file, df in sales_data.items():
    print(f"{file}: {df.shape}")

Loading sales_202106.csv...
Loading sales_202107.csv...
Loading sales_202108.csv...
Loading sales_202109.csv...
Loading sales_202110.csv...
Loading sales_202111.csv...
Loading sales_202112.csv...
sales_202106.csv: (551, 19)
sales_202107.csv: (626, 19)
sales_202108.csv: (661, 19)
sales_202109.csv: (485, 21)
sales_202110.csv: (469, 21)
sales_202111.csv: (344, 19)
sales_202112.csv: (351, 19)


In [71]:
#sales1.2
required_columns = [
    'Transaction Date',
    'Transaction Type',
    'Buyer Country',
    'Sku Id',
    'Amount (Merchant Currency)'
]

cleaned_sales_data = {}

for file, df in sales_data.items():

    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"{file} missing: {missing_columns}")
        continue

    print(f"{file} good")

    df = df[df['Transaction Type'] == 'Charge']

    df = df[df['Sku Id'].isin(['premium', 'unlockcharactermanager'])]

    df = df.dropna(subset=required_columns)

    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

    cleaned_sales_data[file] = df

for file, df in cleaned_sales_data.items():
    print(f"{file}: {len(df)} rows")

sales_202106.csv good
sales_202107.csv good
sales_202108.csv good
sales_202109.csv good
sales_202110.csv good
sales_202111.csv missing: ['Transaction Date', 'Transaction Type', 'Buyer Country', 'Sku Id', 'Amount (Merchant Currency)']
sales_202112.csv missing: ['Transaction Date', 'Transaction Type', 'Buyer Country', 'Sku Id', 'Amount (Merchant Currency)']
sales_202106.csv: 271 rows
sales_202107.csv: 324 rows
sales_202108.csv: 316 rows
sales_202109.csv: 234 rows
sales_202110.csv: 221 rows


In [72]:
#sales1.3
sales_per_day = pd.DataFrame()

for file, df in cleaned_sales_data.items():
    daily_sales = df.groupby(["Transaction Date", "Sku Id"]).agg(
        Total_Sales=("Amount (Merchant Currency)", "sum"),
        Transaction_Count=("Amount (Merchant Currency)", "count")  #do we need it? Yes.
    ).reset_index()

    sales_per_day = pd.concat([sales_per_day, daily_sales], ignore_index=True)

sales_per_day = sales_per_day.groupby(["Transaction Date", "Sku Id"]).agg(
    Total_Sales=("Total_Sales", "sum"),
    Transaction_Count=("Transaction_Count", "sum")
).reset_index()

print(sales_per_day.head())

  Transaction Date                  Sku Id  Total_Sales  Transaction_Count
0       2021-06-01                 premium         4.90                  2
1       2021-06-01  unlockcharactermanager         6.12                  2
2       2021-06-02                 premium        17.23                  6
3       2021-06-02  unlockcharactermanager        20.84                  5
4       2021-06-03                 premium         8.64                  3


In [73]:
#crashes/rating 2.1
crashes_files = [
    "stats_crashes_202106_overview.csv", "stats_crashes_202107_overview.csv",
    "stats_crashes_202108_overview.csv", "stats_crashes_202109_overview.csv",
    "stats_crashes_202110_overview.csv", "stats_crashes_202111_overview.csv",
    "stats_crashes_202112_overview.csv"
]

def load_crashes(file_list):
    dataframes = []
    for file in file_list:
        try:
            print(f"{file}")
            df = pd.read_csv(file, encoding="utf-16", skip_blank_lines=True)
            dataframes.append(df)
        except Exception as e:
            print(f"F{file}: {e}")
    return pd.concat(dataframes, ignore_index=True)

crashes_data = load_crashes(crashes_files)

print(crashes_data.head())



stats_crashes_202106_overview.csv
stats_crashes_202107_overview.csv
stats_crashes_202108_overview.csv
stats_crashes_202109_overview.csv
stats_crashes_202110_overview.csv
stats_crashes_202111_overview.csv
stats_crashes_202112_overview.csv
         Date                         Package Name  Daily Crashes  Daily ANRs
0  2021-06-01  com.vansteinengroentjes.apps.ddfive             15           1
1  2021-06-02  com.vansteinengroentjes.apps.ddfive             12           1
2  2021-06-03  com.vansteinengroentjes.apps.ddfive             20           1
3  2021-06-04  com.vansteinengroentjes.apps.ddfive             13           0
4  2021-06-05  com.vansteinengroentjes.apps.ddfive             14           0


In [74]:
#crashes/rating 2.2
ratings_files = [
    "stats_ratings_202106_overview.csv", "stats_ratings_202107_overview.csv",
    "stats_ratings_202108_overview.csv", "stats_ratings_202109_overview.csv",
    "stats_ratings_202110_overview.csv", "stats_ratings_202111_overview.csv",
    "stats_ratings_202112_overview.csv"
]

def load_ratings(file_list):
    dataframes = []
    for file in file_list:
        try:
            print(f"{file}")
            df = pd.read_csv(file, encoding="utf-16", skip_blank_lines=True)
            dataframes.append(df)
        except Exception as e:
            print(f"F{file}: {e}")
    return pd.concat(dataframes, ignore_index=True)

ratings_data = load_ratings(ratings_files)

print(ratings_data.head())

stats_ratings_202106_overview.csv
stats_ratings_202107_overview.csv
stats_ratings_202108_overview.csv
stats_ratings_202109_overview.csv
stats_ratings_202110_overview.csv
stats_ratings_202111_overview.csv
stats_ratings_202112_overview.csv
         Date                         Package Name  Daily Average Rating  \
0  2021-06-01  com.vansteinengroentjes.apps.ddfive                   5.0   
1  2021-06-02  com.vansteinengroentjes.apps.ddfive                   2.0   
2  2021-06-03  com.vansteinengroentjes.apps.ddfive                   NaN   
3  2021-06-04  com.vansteinengroentjes.apps.ddfive                   NaN   
4  2021-06-05  com.vansteinengroentjes.apps.ddfive                   NaN   

   Total Average Rating  
0                  4.15  
1                  4.15  
2                  4.15  
3                  4.15  
4                  4.15  


In [75]:
#crashes/ratings 2.3
required_ratings_columns = ['Date', 'Daily Average Rating']
filtered_ratings_data = ratings_data[required_ratings_columns].dropna()

required_crashes_columns = ['Date', 'Daily Crashes', 'Daily ANRs']
filtered_crashes_data = crashes_data[required_crashes_columns].dropna()

merged_data = pd.merge(
    filtered_crashes_data,
    filtered_ratings_data,
    on='Date',
    how='inner'
)

print("good job me:")
print(merged_data.head())

good job me:
         Date  Daily Crashes  Daily ANRs  Daily Average Rating
0  2021-06-01             15           1                   5.0
1  2021-06-02             12           1                   2.0
2  2021-06-07              4           0                   5.0
3  2021-06-08             14           0                   5.0
4  2021-06-12              8           1                   4.0


In [76]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

output_notebook()

# 1: Sales Total
sales_total_source = ColumnDataSource(data=sales_per_day)

# 2: Premium SkuId
sales_per_day_premium = sales_per_day[sales_per_day['Sku Id'] == 'premium']
sales_premium_source = ColumnDataSource(data=sales_per_day_premium)

# 3: UnlockCharacterManager SkuId
sales_per_day_unlock = sales_per_day[sales_per_day['Sku Id'] == 'unlockcharactermanager']
sales_unlock_source = ColumnDataSource(data=sales_per_day_unlock)

# 4+5: Crashes/Ratings Data
ratings_crashes_source = ColumnDataSource(data=merged_data)
ratings_anrs_source = ColumnDataSource(data=merged_data)

#Total Sales/Day
p1 = figure(
    title="Total Sales Per Day",
    x_axis_type="datetime",
    width=800,
    height=400,
    tools="pan,box_zoom,reset,save",
)
p1.line(
    x='Transaction Date',
    y='Total_Sales',
    source=sales_total_source,
    line_width=2,
    color="blue",
    legend_label="Total Sales"
)
p1.line(
    x='Transaction Date',
    y='Transaction_Count',
    source=sales_total_source,
    line_width=2,
    color="orange",
    legend_label="Transaction Count"
)
p1.legend.location = "top_left"
p1.xaxis.axis_label = "Date"
p1.yaxis.axis_label = "Values"

#Premium SkuId Sales/Day
p2 = figure(
    title="Sales Volume for SKU: Premium",
    x_axis_type="datetime",
    width=800,
    height=400,
    tools="pan,box_zoom,reset,save",
)
p2.line(
    x='Transaction Date',
    y='Total_Sales',
    source=sales_premium_source,
    line_width=2,
    color="teal",
    legend_label="Premium Sales (€)"
)
p2.line(
    x='Transaction Date',
    y='Transaction_Count',
    source=sales_premium_source,
    line_width=2,
    color="pink",
    legend_label="Transaction Count"
)
p2.legend.location = "top_left"
p2.xaxis.axis_label = "Date"
p2.yaxis.axis_label = "Values"

#UnlockCharacterManager SkuId Sales/Day
p3 = figure(
    title="Sales Volume for SKU: UnlockCharacterManager",
    x_axis_type="datetime",
    width=800,
    height=400,
    tools="pan,box_zoom,reset,save",
)
p3.line(
    x='Transaction Date',
    y='Total_Sales',
    source=sales_unlock_source,
    line_width=2,
    color="indigo",
    legend_label="UnlockCharacterManager Sales (€)"
)
p3.line(
    x='Transaction Date',
    y='Transaction_Count',
    source=sales_unlock_source,
    line_width=2,
    color="orange",
    legend_label="Transaction Count"
)
p3.legend.location = "top_left"
p3.xaxis.axis_label = "Date"
p3.yaxis.axis_label = "Values"

#Daily Crashes/Avg Rating
p4 = figure(
    title="Daily Crashes vs Average Rating",
    x_axis_label="Daily Average Rating",
    y_axis_label="Daily Crashes",
    height=400,
    width=800,
    tools="pan,box_zoom,reset,save",
)
p4.scatter(
    x="Daily Average Rating",
    y="Daily Crashes",
    source=ratings_crashes_source,
    color="blue",
    size=8,
    legend_label="Crashes"
)
p4.legend.location = "top_right"

#Daily ANRs/Avg Rating
p5 = figure(
    title="Daily ANRs vs Average Rating",
    x_axis_label="Daily Average Rating",
    y_axis_label="Daily ANRs",
    height=400,
    width=800,
    tools="pan,box_zoom,reset,save",
)
p5.scatter(
    x="Daily Average Rating",
    y="Daily ANRs",
    source=ratings_anrs_source,
    color="orange",
    size=8,
    legend_label="ANRs"
)
p5.legend.location = "top_right"

# **Combine All Charts**
layout = gridplot([[p1], [p2], [p3], [p4], [p5]])
show(layout)

In [81]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select, CustomJS
from bokeh.layouts import column
from bokeh.io import output_notebook

output_notebook()

combined_sales_data = pd.concat(cleaned_sales_data.values(), ignore_index=True)

combined_sales_data['Transaction Date'] = pd.to_datetime(combined_sales_data['Transaction Date'])

combined_sales_data['Transaction Month'] = combined_sales_data['Transaction Date'].dt.to_period('M').dt.to_timestamp()
sales_per_month_country = combined_sales_data.groupby(['Transaction Month', 'Buyer Country']) \
    .agg({'Amount (Merchant Currency)': 'sum'}).reset_index()

sales_per_month_country.rename(columns={
    'Transaction Month': 'Transaction_Month',
    'Buyer Country': 'Buyer_Country',
    'Amount (Merchant Currency)': 'Total_Sales'
}, inplace=True)

country_data = {}
for country in sales_per_month_country['Buyer_Country'].unique():
    country_df = sales_per_month_country[sales_per_month_country['Buyer_Country'] == country]
    country_data[country] = {
        'Transaction_Month': country_df['Transaction_Month'],
        'Total_Sales': country_df['Total_Sales']
    }

default_country = list(country_data.keys())[0]
default_data = country_data[default_country]

source = ColumnDataSource(data=default_data)

p = figure(
    title=f"Total Sales Per Month for {default_country}",
    x_axis_type="datetime",
    width=800,
    height=400,
    tools="pan,box_zoom,reset,save"
)
p.line(x='Transaction_Month', y='Total_Sales', source=source, line_width=2, color="blue", legend_label="Total Sales (€)")
p.circle(x='Transaction_Month', y='Total_Sales', source=source, size=6, color="blue")
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Total Sales (€)"
p.legend.location = "top_left"

menu = Select(title="Select Country:", value=default_country, options=list(country_data.keys()))

callback = CustomJS(args=dict(source=source, data=country_data, plot=p), code="""
    const selected_country = cb_obj.value;
    const new_data = data[selected_country];
    source.data = new_data;
    plot.title.text = `Total Sales Per Month for ${selected_country}`;
    source.change.emit();
""")
menu.js_on_change('value', callback)

layout = column(menu, p)
show(layout)